# 1. Installing Dependencies

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import sklearn

# 2. Detect landmarks using MediaPipe

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities


In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoints


In [ ]:
# Extracting keypoints. Get all the landmarks for pose, face, left and right hands, in four seperate errays
# if no values, replace with blank array
# x,y,z coordinates hence multiply the landmarks by 3

# Concatenate into one array

def extract_keypoints(results):
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return rh

# 4. Setup Folders for collection

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('C:/Users/hassa/Desktop/MP_Data') 

# Actions that we try to detect
actions = np.array(['alif', 'bay', 'pay', 'tay', 'ttay', 'say', 'jeem', 'chay', 'hay', 'khay', 'dal', 
           'daal', 'zal', 'ray', 'rhay', 'zay', 'seen', 'sheen', 'swadh', 'zwad', 'toin', 'zoin', 
           'ghain', 'ain', 'fay', 'kaaf', 'kiaf', 'gaaf', 'laam','meem','noon','wow','choti_haay','barihaay',
           'choti_yay','bari_yay','hamza'])


# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 1

In [ ]:
for action in actions: 
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect keypoints, save as .npy

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(0)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# 6. Preprocess Data, Creating labels and features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}


In [ ]:
label_map

{'alif': 0,
 'bay': 1,
 'pay': 2,
 'tay': 3,
 'ttay': 4,
 'say': 5,
 'jeem': 6,
 'chay': 7,
 'hay': 8,
 'khay': 9,
 'dal': 10,
 'daal': 11,
 'zal': 12,
 'ray': 13,
 'rhay': 14,
 'zay': 15,
 'seen': 16,
 'sheen': 17,
 'swadh': 18,
 'zwad': 19,
 'toin': 20,
 'zoin': 21,
 'ghain': 22,
 'ain': 23,
 'fay': 24,
 'kaaf': 25,
 'kiaf': 26,
 'gaaf': 27,
 'laam': 28,
 'meem': 29,
 'noon': 30,
 'wow': 31,
 'choti_haay': 32,
 'barihaay': 33,
 'choti_yay': 34,
 'bari_yay': 35,
 'hamza': 36}

In [ ]:
#

sequences, labels = [], []      # 
for action in actions:          # Loop through each action
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []             # Represents all frames for that sequence
        
        # Grabs all frames, append to window. 
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

(1110, 30, 63)

In [ ]:
np.array(labels).shape


(1110,)

In [ ]:
X = np.array(sequences)


In [ ]:
X.shape

(1110, 30, 63)

In [ ]:
X


array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 1.36166096e-01,  7.02629626e-01, -3.30804767e-07, ...,
          1.13871992e-01,  5.92121899e-01, -1.62987560e-02],
        [ 1.35152876e-01,  7.02232838e-01, -3.33899749e-07, ...,
          1.13750622e-01,  5.92297733e-01, -1.51789160e-02],
        [ 1.34203553e-01,  7.00679719e-01, -3.04482029e-07, ...,
          1.12168305e-01,  5.89823246e-01, -1.70175415e-02]],

       [[ 2.22813696e-01,  7.50495136e-01, -2.64211565e-07, ...,
          2.06181094e-01,  6.50117278e-01, -2.63720416e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e

In [ ]:
# Converted into a binary flag

y = to_categorical(labels).astype(int)


In [ ]:
y


array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
y_test.shape


(56, 37)

In [ ]:
y_train.shape

(1054, 37)

In [ ]:
X_train.shape


(1054, 30, 63)

In [ ]:
X_test.shape

(56, 30, 63)

# Build LSTM model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

In [ ]:
# Set up Tensorboard, to see logs while training

log_dir = os.path.join(r'C:\Users\hassa\Desktop\ICV Project\2')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
actions.shape[0]

37

In [ ]:
checkpoint_filepath = os.path.join(r'C:\Users\hassa\Desktop\ICV Project\file')

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath ,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_freq = 4,
    save_best_only=True)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='sigmoid', input_shape=(30,63)))

model.add(LSTM(256, return_sequences=True, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=False, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(actions.shape[0], activation='softmax'))


# model = Sequential()
# model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,63)))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])


Epoch 1/2000
33/33 [==============================] - 5s 71ms/step - loss: 3.6714 - categorical_accuracy: 0.0199
Epoch 2/2000
33/33 [==============================] - 2s 72ms/step - loss: 3.6367 - categorical_accuracy: 0.0218
Epoch 3/2000
33/33 [==============================] - 2s 72ms/step - loss: 3.6274 - categorical_accuracy: 0.0304
Epoch 4/2000
33/33 [==============================] - 2s 72ms/step - loss: 3.6241 - categorical_accuracy: 0.0114
Epoch 5/2000
33/33 [==============================] - 3s 77ms/step - loss: 3.6203 - categorical_accuracy: 0.0209
Epoch 6/2000
33/33 [==============================] - 3s 76ms/step - loss: 3.6232 - categorical_accuracy: 0.0304
Epoch 7/2000
33/33 [==============================] - 3s 77ms/step - loss: 3.6180 - categorical_accuracy: 0.0256
Epoch 8/2000
33/33 [==============================] - 3s 80ms/step - loss: 3.6194 - categorical_accuracy: 0.0228
Epoch 9/2000
33/33 [==============================] - 2s 76ms/step - loss: 3.6218 - categorical_

KeyboardInterrupt: 

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            32768     
                                                                 
 lstm_4 (LSTM)               (None, 30, 256)           328704    
                                                                 
 dropout (Dropout)           (None, 30, 256)           0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                82176     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)               

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)

2/2 [==============================] - 0s 19ms/step


In [ ]:
actions[np.argmax(res[1])]


'meem'

In [ ]:
actions[np.argmax(y_test[1])]


'meem'

# 9. Save Weights


In [ ]:
model.save('C:/Users/hassa/Desktop/ICV Project/actions_9.h5')


In [ ]:
model.load_weights('C:/Users/hassa/Desktop/ICV Project/actions_9.h5')

# 10. Evaluation using confusion matrix

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score


In [ ]:
yhat = model.predict(X_train)

33/33 [==============================] - 1s 24ms/step


In [ ]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)


array([[[1025,    1],
        [   1,   27]],

       [[1026,    0],
        [   0,   28]],

       [[1024,    0],
        [   2,   28]],

       [[1025,    0],
        [   0,   29]],

       [[ 997,   28],
        [   1,   28]],

       [[1025,    0],
        [   0,   29]],

       [[1026,    0],
        [   0,   28]],

       [[1026,    0],
        [   0,   28]],

       [[ 995,   30],
        [   0,   29]],

       [[1026,    0],
        [   0,   28]],

       [[1027,    0],
        [   0,   27]],

       [[1026,    0],
        [  28,    0]],

       [[1025,    0],
        [   0,   29]],

       [[1026,    0],
        [   0,   28]],

       [[1025,    0],
        [   0,   29]],

       [[1025,    0],
        [  29,    0]],

       [[1024,    1],
        [   1,   28]],

       [[1024,    0],
        [   0,   30]],

       [[1026,    1],
        [   0,   27]],

       [[1026,    0],
        [   2,   26]],

       [[ 996,   28],
        [   0,   30]],

       [[1024,    0],
        [   

In [ ]:
accuracy_score(ytrue, yhat)


0.9089184060721063

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.6

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-30:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti